In [1]:
import pandas as pd
import dask.dataframe as dd

In [2]:
d = [
    {
        'id':[2],
        'object_names':['bab'],
        'paths':['/bab'],
        'size':[12],
        'type':['zip'],
        'upload':[True]
    },
    {
        'id':[1],
        'object_names':['bob'],
        'paths':['/bob'],
        'size':[2],
        'type':['zip'],
        'upload':[False]
    },
    {
        'id':[0],
        'object_names':['bib'],
        'paths':['/bib'],
        'size':[9],
        'type':['file'],
        'upload':[True]
        }
]

In [3]:
ddf = dd.from_pandas(pd.DataFrame([], columns=['id', 'object_names', 'paths', 'size', 'type', 'upload']))

In [4]:
for de in d:
    df = pd.DataFrame(de)
    if (len(ddf.index) == 0):
        ddf = dd.from_pandas(df, npartitions=1)
    else:
        ddf = dd.concat([ddf, dd.from_pandas(df, npartitions=1)])

In [5]:
ddf

,id,object_names,paths,size,type,upload
npartitions=3,,,,,,
,int64,string,string,int64,string,bool
,...,...,...,...,...,...
,...,...,...,...,...,...
,...,...,...,...,...,...


In [6]:
del d[2]

In [7]:
d

[{'id': [2],
  'object_names': ['bab'],
  'paths': ['/bab'],
  'size': [12],
  'type': ['zip'],
  'upload': [True]},
 {'id': [1],
  'object_names': ['bob'],
  'paths': ['/bob'],
  'size': [2],
  'type': ['zip'],
  'upload': [False]}]

In [8]:
ddf.compute()

,id,object_names,paths,size,type,upload
0,2,bab,/bab,12,zip,True
0,1,bob,/bob,2,zip,False
0,0,bib,/bib,9,file,True


In [9]:
import os

In [10]:
path = os.path.abspath('../../../test_tree_large')

In [11]:
path

'/home/dave/test_tree_large'

In [12]:
df = pd.DataFrame()
folder_count = 0
file_count = 0
for folder, _, files in os.walk(path, topdown=True):
    if len(files) > 0:
        folder_count += 1
        file_count += len(files)
        # print(f'folder: {folder}')
        # print(f'sub_folders: {sub_folders}')
        # print(f'files: {files}')
        folder_files = [os.path.join(folder, file) for file in files]
        df = pd.concat([df, pd.DataFrame(folder_files, columns=['local_path'])], ignore_index=True)

In [13]:
df['size'] = df['local_path'].apply(lambda x: os.path.getsize(x))
df['object_names'] = df['local_path'].apply(lambda x: os.path.basename(x))
df['upload'] = False
df['type'] = 'file'

In [14]:
ddf = dd.from_pandas(df, npartitions=100)
ddf

,local_path,size,object_names,upload,type
npartitions=100,,,,,
0,string,int64,string,bool,string
91,...,...,...,...,...
...,...,...,...,...,...
8913,...,...,...,...,...
9002,...,...,...,...,...


In [15]:
current_objects = pd.Series(os.listdir('../../../test_tree_large/7/'))

In [16]:
current_objects = pd.DataFrame(current_objects, columns=['object_names'])
current_objects

,object_names
0,7_256.f
1,7_175.f
2,7_609.f
3,7_769.f
4,7_422.f
...,...
996,7_834.f
997,7_845.f
998,7_195.f
999,7_507.f


In [17]:
ddf['exists'] = ddf['object_names'].isin(current_objects['object_names'])

In [18]:
done = ddf.compute()

In [19]:
done

,local_path,size,object_names,upload,type,exists
0,/home/dave/test_tree_large/test-test_tree_larg...,96,test-test_tree_large-lsst-backup.csv,False,file,False
1,/home/dave/test_tree_large/test_tree_traversal...,276,test_tree_traversal.yaml,False,file,False
2,/home/dave/test_tree_large/test.csv,334208,test.csv,False,file,False
3,/home/dave/test_tree_large/3/3_425.f,1048576,3_425.f,False,file,False
4,/home/dave/test_tree_large/3/3_805.f,1048576,3_805.f,False,file,False
...,...,...,...,...,...,...
8998,/home/dave/test_tree_large/1/1_632.f,1048576,1_632.f,False,file,False
8999,/home/dave/test_tree_large/1/1_686.f,1048576,1_686.f,False,file,False
9000,/home/dave/test_tree_large/1/1_671.f,1048576,1_671.f,False,file,False
9001,/home/dave/test_tree_large/1/1_613.f,1048576,1_613.f,False,file,False


In [20]:
uploads = done[done['exists'] == False].drop(columns=['exists'])

In [21]:
uploads

,local_path,size,object_names,upload,type
0,/home/dave/test_tree_large/test-test_tree_larg...,96,test-test_tree_large-lsst-backup.csv,False,file
1,/home/dave/test_tree_large/test_tree_traversal...,276,test_tree_traversal.yaml,False,file
2,/home/dave/test_tree_large/test.csv,334208,test.csv,False,file
3,/home/dave/test_tree_large/3/3_425.f,1048576,3_425.f,False,file
4,/home/dave/test_tree_large/3/3_805.f,1048576,3_805.f,False,file
...,...,...,...,...,...
8998,/home/dave/test_tree_large/1/1_632.f,1048576,1_632.f,False,file
8999,/home/dave/test_tree_large/1/1_686.f,1048576,1_686.f,False,file
9000,/home/dave/test_tree_large/1/1_671.f,1048576,1_671.f,False,file
9001,/home/dave/test_tree_large/1/1_613.f,1048576,1_613.f,False,file


In [22]:
max_zip_size = 1048576 * 100

In [23]:
uploads

,local_path,size,object_names,upload,type
0,/home/dave/test_tree_large/test-test_tree_larg...,96,test-test_tree_large-lsst-backup.csv,False,file
1,/home/dave/test_tree_large/test_tree_traversal...,276,test_tree_traversal.yaml,False,file
2,/home/dave/test_tree_large/test.csv,334208,test.csv,False,file
3,/home/dave/test_tree_large/3/3_425.f,1048576,3_425.f,False,file
4,/home/dave/test_tree_large/3/3_805.f,1048576,3_805.f,False,file
...,...,...,...,...,...
8998,/home/dave/test_tree_large/1/1_632.f,1048576,1_632.f,False,file
8999,/home/dave/test_tree_large/1/1_686.f,1048576,1_686.f,False,file
9000,/home/dave/test_tree_large/1/1_671.f,1048576,1_671.f,False,file
9001,/home/dave/test_tree_large/1/1_613.f,1048576,1_613.f,False,file


In [24]:
import numpy as np

In [25]:

uploads['individual_upload'] = np.random.choice([True, False], size=len(uploads))
uploads

,local_path,size,object_names,upload,type,individual_upload
0,/home/dave/test_tree_large/test-test_tree_larg...,96,test-test_tree_large-lsst-backup.csv,False,file,False
1,/home/dave/test_tree_large/test_tree_traversal...,276,test_tree_traversal.yaml,False,file,False
2,/home/dave/test_tree_large/test.csv,334208,test.csv,False,file,False
3,/home/dave/test_tree_large/3/3_425.f,1048576,3_425.f,False,file,False
4,/home/dave/test_tree_large/3/3_805.f,1048576,3_805.f,False,file,True
...,...,...,...,...,...,...
8998,/home/dave/test_tree_large/1/1_632.f,1048576,1_632.f,False,file,True
8999,/home/dave/test_tree_large/1/1_686.f,1048576,1_686.f,False,file,False
9000,/home/dave/test_tree_large/1/1_671.f,1048576,1_671.f,False,file,False
9001,/home/dave/test_tree_large/1/1_613.f,1048576,1_613.f,False,file,False


In [26]:
len(uploads)

8003

In [27]:

cumulative_size = 0
batches = [None]
batch = [None]
batch_number = [None]
for row in uploads.iterrows():
    if row[1]['individual_upload']:
        batch_number.append(0)
    else:
        size = row[1]['size']
        if cumulative_size + size > max_zip_size:
            batches.append(1)
            batch = [1]
            cumulative_size = size
        else:
            batch.append(1)
            cumulative_size += size
            print(row[1].name)
            if row[1].name == uploads.index[-1]:
                print('inif')
                batches.append(1)
        batch_number.append(len(batches))
    # print(f'row: {row[1].name} - batch: {len(batches)} - cumulative_size: {cumulative_size} - size: {size} - individual_upload: {row[1]["individual_upload"]}')
    # k+=1
    # if k == 10:
        # break

batch_number[-10:]

0
1
2
3
5
6
7
9
11
12
13
17
20
22
25
27
31
33
34
36
40
43
45
47
48
49
54
55
56
57
59
60
61
62
63
64
67
68
70
71
72
73
75
78
79
80
84
87
89
92
94
95
97
98
99
100
101
105
106
108
112
113
114
115
118
120
122
124
125
126
129
130
131
134
141
142
146
151
154
158
159
161
162
163
167
169
172
174
175
176
179
180
181
182
183
188
191
192
193
194
196
198
200
205
207
208
211
212
216
218
219
220
223
226
230
231
236
238
239
241
242
243
244
246
247
248
251
253
254
257
258
259
260
265
266
271
273
274
275
277
281
282
284
286
287
288
289
290
292
294
295
296
300
303
304
306
308
309
310
313
319
320
321
322
323
325
326
328
329
332
333
334
335
336
337
343
348
349
353
354
355
356
358
359
360
361
362
363
364
366
367
368
370
371
373
374
375
378
380
381
383
387
391
393
394
395
396
400
402
403
405
406
407
409
410
411
412
414
415
417
420
422
423
424
425
426
429
430
435
438
441
448
452
454
456
460
463
464
465
466
467
468
469
471
473
474
475
476
479
480
481
484
485
487
488
490
491
493
501
505
506
511
512
514
524
525

[40, 0, 0, 0, 40, 0, 40, 40, 40, 41]

In [28]:
len(batch_number)

8004

In [29]:
uploads['batch_number'] = batch_number[1:]
uploads

,local_path,size,object_names,upload,type,individual_upload,batch_number
0,/home/dave/test_tree_large/test-test_tree_larg...,96,test-test_tree_large-lsst-backup.csv,False,file,False,1
1,/home/dave/test_tree_large/test_tree_traversal...,276,test_tree_traversal.yaml,False,file,False,1
2,/home/dave/test_tree_large/test.csv,334208,test.csv,False,file,False,1
3,/home/dave/test_tree_large/3/3_425.f,1048576,3_425.f,False,file,False,1
4,/home/dave/test_tree_large/3/3_805.f,1048576,3_805.f,False,file,True,0
...,...,...,...,...,...,...,...
8998,/home/dave/test_tree_large/1/1_632.f,1048576,1_632.f,False,file,True,0
8999,/home/dave/test_tree_large/1/1_686.f,1048576,1_686.f,False,file,False,40
9000,/home/dave/test_tree_large/1/1_671.f,1048576,1_671.f,False,file,False,40
9001,/home/dave/test_tree_large/1/1_613.f,1048576,1_613.f,False,file,False,40


In [32]:
print(uploads[uploads['batch_number'] == 1]['size'].sum() < max_zip_size)

True


In [33]:
for bn in range(1, uploads['batch_number'].max()+1):
    size = uploads[uploads['batch_number'] == bn]['size'].sum()
    count = len(uploads[uploads['batch_number'] == bn])
    print(f'Batch Number: {bn}, Size: {size}, Count: {count}')
    if size > max_zip_size:
        print('Size is greater than max zip size')
    else:
        print('Size is less than max zip size')

Batch Number: 1, Size: 104143604, Count: 102
Size is less than max zip size
Batch Number: 2, Size: 104857600, Count: 100
Size is less than max zip size
Batch Number: 3, Size: 104857600, Count: 100
Size is less than max zip size
Batch Number: 4, Size: 104857600, Count: 100
Size is less than max zip size
Batch Number: 5, Size: 104857600, Count: 100
Size is less than max zip size
Batch Number: 6, Size: 104857600, Count: 100
Size is less than max zip size
Batch Number: 7, Size: 104857600, Count: 100
Size is less than max zip size
Batch Number: 8, Size: 104857600, Count: 100
Size is less than max zip size
Batch Number: 9, Size: 104857600, Count: 100
Size is less than max zip size
Batch Number: 10, Size: 104857600, Count: 100
Size is less than max zip size
Batch Number: 11, Size: 104857600, Count: 100
Size is less than max zip size
Batch Number: 12, Size: 104857600, Count: 100
Size is less than max zip size
Batch Number: 13, Size: 104857600, Count: 100
Size is less than max zip size
Batch Nu

In [34]:
uploads

,local_path,size,object_names,upload,type,individual_upload,batch_number
0,/home/dave/test_tree_large/test-test_tree_larg...,96,test-test_tree_large-lsst-backup.csv,False,file,False,1
1,/home/dave/test_tree_large/test_tree_traversal...,276,test_tree_traversal.yaml,False,file,False,1
2,/home/dave/test_tree_large/test.csv,334208,test.csv,False,file,False,1
3,/home/dave/test_tree_large/3/3_425.f,1048576,3_425.f,False,file,False,1
4,/home/dave/test_tree_large/3/3_805.f,1048576,3_805.f,False,file,True,0
...,...,...,...,...,...,...,...
8998,/home/dave/test_tree_large/1/1_632.f,1048576,1_632.f,False,file,True,0
8999,/home/dave/test_tree_large/1/1_686.f,1048576,1_686.f,False,file,False,40
9000,/home/dave/test_tree_large/1/1_671.f,1048576,1_671.f,False,file,False,40
9001,/home/dave/test_tree_large/1/1_613.f,1048576,1_613.f,False,file,False,40


In [72]:
zips = uploads[uploads['batch_number'] > 0][['batch_number']].drop_duplicates()

In [73]:
zips['path'] = uploads[uploads['batch_number'] > 0]['local_path'].groupby(uploads['batch_number']).agg(list_aggredator).values
zips['size'] = uploads[uploads['batch_number'] > 0]['size'].groupby(uploads['batch_number']).sum().values
zips['object_names'] = uploads[uploads['batch_number'] > 0]['object_names'].groupby(uploads['batch_number']).agg(list_aggredator).values
zips['upload'] = True
zips['uploaded'] = False
zips['type'] = 'zip'
zips['id'] = zips['batch_number']
zips = zips.drop(columns=['batch_number'])
zips

,path,size,object_names,upload,uploaded,type,id
1,/home/dave/test_tree_large/test_tree_traversal...,103809300,test_tree_traversal.yaml|3_425.f|3_805.f|3_238...,True,False,zip,1
223,/home/dave/test_tree_large/3/3_274.f|/home/dav...,104857600,3_274.f|3_411.f|3_905.f|3_5.f|3_287.f|3_463.f|...,True,False,zip,2
425,/home/dave/test_tree_large/3/3_559.f|/home/dav...,104857600,3_559.f|3_51.f|3_740.f|3_233.f|3_823.f|3_942.f...,True,False,zip,3
634,/home/dave/test_tree_large/3/3_296.f|/home/dav...,104857600,3_296.f|3_27.f|3_676.f|3_150.f|3_85.f|3_370.f|...,True,False,zip,4
812,/home/dave/test_tree_large/3/3_558.f|/home/dav...,104857600,3_558.f|3_854.f|3_847.f|3_934.f|3_346.f|3_245....,True,False,zip,5
998,/home/dave/test_tree_large/3/3_421.f|/home/dav...,104857600,3_421.f|3_956.f|5_944.f|5_735.f|5_969.f|5_630....,True,False,zip,6
2214,/home/dave/test_tree_large/5/5_616.f|/home/dav...,104857600,5_616.f|5_730.f|5_417.f|5_27.f|5_793.f|5_731.f...,True,False,zip,7
2405,/home/dave/test_tree_large/5/5_66.f|/home/dave...,104857600,5_66.f|5_403.f|5_440.f|5_268.f|5_815.f|5_152.f...,True,False,zip,8
2627,/home/dave/test_tree_large/5/5_94.f|/home/dave...,104857600,5_94.f|5_37.f|5_190.f|5_608.f|5_864.f|5_705.f|...,True,False,zip,9
2847,/home/dave/test_tree_large/5/5_578.f|/home/dav...,104857600,5_578.f|5_841.f|5_644.f|5_902.f|5_748.f|5_540....,True,False,zip,10


In [74]:
ind_files = uploads[uploads['batch_number'] == 0].drop(columns=['batch_number', 'individual_upload'])
ind_files['upload'] = True
ind_files['uploaded'] = False
ind_files.rename(columns={'local_path': 'path'}, inplace=True)
ind_files['id'] = None
ind_files

,path,size,object_names,upload,type,uploaded,id
0,/home/dave/test_tree_large/test-test_tree_larg...,96,test-test_tree_large-lsst-backup.csv,True,file,False,None
2,/home/dave/test_tree_large/test.csv,334208,test.csv,True,file,False,None
5,/home/dave/test_tree_large/3/3_795.f,1048576,3_795.f,True,file,False,None
7,/home/dave/test_tree_large/3/3_562.f,1048576,3_562.f,True,file,False,None
10,/home/dave/test_tree_large/3/3_802.f,1048576,3_802.f,True,file,False,None
...,...,...,...,...,...,...,...
8996,/home/dave/test_tree_large/1/1_314.f,1048576,1_314.f,True,file,False,None
8997,/home/dave/test_tree_large/1/1_654.f,1048576,1_654.f,True,file,False,None
8998,/home/dave/test_tree_large/1/1_632.f,1048576,1_632.f,True,file,False,None
8999,/home/dave/test_tree_large/1/1_686.f,1048576,1_686.f,True,file,False,None


In [35]:
uploads

,local_path,size,object_names,upload,type,individual_upload,batch_number
0,/home/dave/test_tree_large/test-test_tree_larg...,96,test-test_tree_large-lsst-backup.csv,False,file,False,1
1,/home/dave/test_tree_large/test_tree_traversal...,276,test_tree_traversal.yaml,False,file,False,1
2,/home/dave/test_tree_large/test.csv,334208,test.csv,False,file,False,1
3,/home/dave/test_tree_large/3/3_425.f,1048576,3_425.f,False,file,False,1
4,/home/dave/test_tree_large/3/3_805.f,1048576,3_805.f,False,file,True,0
...,...,...,...,...,...,...,...
8998,/home/dave/test_tree_large/1/1_632.f,1048576,1_632.f,False,file,True,0
8999,/home/dave/test_tree_large/1/1_686.f,1048576,1_686.f,False,file,False,40
9000,/home/dave/test_tree_large/1/1_671.f,1048576,1_671.f,False,file,False,40
9001,/home/dave/test_tree_large/1/1_613.f,1048576,1_613.f,False,file,False,40


In [36]:
from dask import dataframe as dd

In [37]:
ddf = dd.from_pandas(uploads, 2)

In [39]:
ddf['local_path'].compute()

0       /home/dave/test_tree_large/test-test_tree_larg...
1       /home/dave/test_tree_large/test_tree_traversal...
2                     /home/dave/test_tree_large/test.csv
3                    /home/dave/test_tree_large/3/3_425.f
4                    /home/dave/test_tree_large/3/3_805.f
                              ...                        
8998                 /home/dave/test_tree_large/1/1_632.f
8999                 /home/dave/test_tree_large/1/1_686.f
9000                 /home/dave/test_tree_large/1/1_671.f
9001                 /home/dave/test_tree_large/1/1_613.f
9002                 /home/dave/test_tree_large/1/1_842.f
Name: local_path, Length: 8003, dtype: string

In [40]:
dd_list_aggregation = dd.Aggregation(
    'list_aggregation',
    list,  # chunks are aggregated into a list of strings
    list,  # strings are aggregated into a list of strings
    lambda x: '|'.join(x) if x.instanceof(list) else x  #, meta=('x', 'object'),
    # '|'.join        # finalize by joining the list of strings
)

In [41]:
# ddf2 = dd.from_pandas(pd.DataFrame.from_dict({'local_path':[], 'batch_number':[]}), 2)
# ddf2 = dd.from_pandas(pd.DataFrame.from_dict(
#     {
#         'local_path': [],
#         'batch_number': []
#     },
# ))
del ddf2

NameError: name 'ddf2' is not defined

In [80]:
ddf2 = ddf[['local_path', 'batch_number']].groupby('batch_number').agg(list)  #.reset_index()
ddf2['local_path'] = ddf2['local_path'].apply(lambda x: '|'.join(x), meta=('x', 'object'))
ddf2.compute()
# ddf2['batch_number'] = ddf['batch_number'].drop_duplicates()
# ddf2 = ddf2.compute()

,local_path
batch_number,
1,/home/dave/test_tree_large/test-test_tree_larg...
0,/home/dave/test_tree_large/3/3_805.f|/home/dav...
2,/home/dave/test_tree_large/3/3_663.f|/home/dav...
3,/home/dave/test_tree_large/3/3_17.f|/home/dave...
4,/home/dave/test_tree_large/3/3_921.f|/home/dav...
5,/home/dave/test_tree_large/3/3_117.f|/home/dav...
6,/home/dave/test_tree_large/3/3_927.f|/home/dav...
7,/home/dave/test_tree_large/5/5_297.f|/home/dav...
8,/home/dave/test_tree_large/5/5_98.f|/home/dave...


In [266]:
ddf2['batch_number'] = ddf2['batch_number'].astype('int')
ddf2

,local_path,batch_number
npartitions=2,,
,float64,int64
,...,...
,...,...


In [267]:
# ddf[ddf['batch_number'] == 1][['local_path', 'batch_number']].groupby('batch_number').agg(list)['local_path'].compute().iloc[0]

In [268]:
ddf2

,local_path,batch_number
npartitions=2,,
,float64,int64
,...,...
,...,...


In [ ]:
#  = ddf[['batch_number','local_path']].groupby(ddf['batch_number'])['local_path'].agg(list)
ddf2['local_path'] = ddf['local_path'].groupby(ddf['batch_number']).agg(list)
# ddf2.compute()
# a = a.apply().compute()
ddf2['local_path'] = ddf2['local_path'].apply(lambda x: '|'.join(x), meta=('x', 'object'))

In [271]:
ddf2.compute()

TypeError: can only join an iterable